In [7]:
import torch
import torch.nn as nn
import math

# LayerNorm

## 核心思想

对每个样本（token）自己的所有特征独立做“均值为 0、标准差为 1”的归一化，完全不依赖 batch 大小。

![LN](pic/LN.png)

## 为什么这么做

由于对每个样本做了归一化，首先带来的好处是梯度是稳定的，不会过大或者过小，而梯度稳定则是训练稳定的重要前提。

In [12]:

class layernorm(nn.Module):
    def __init__(self, input_dim, eps=1e-5) -> None:
        super().__init__()
        self.eps = eps
        self.input_dim = input_dim
        self.weight = nn.Parameter(torch.ones(self.input_dim))
        self.bias = nn.Parameter(torch.zeros(self.input_dim))

    def forward(self, x):
        mean = x.mean(dim= -1, keepdim=True)
        var = x.var(dim= -1, keepdim=True, unbiased=False)
        x_norm = (x - mean) / torch.sqrt(var + self.eps)
        return self.weight * x_norm + self.bias

## 代码测试

In [13]:
x = torch.tensor([[1., 2., 3., 4.], [10., 20., 30., 40.]], dtype=torch.float32)
ln = layernorm(4)
x_norm = ln(x)
x_norm

tensor([[-1.3416, -0.4472,  0.4472,  1.3416],
        [-1.3416, -0.4472,  0.4472,  1.3416]], grad_fn=<AddBackward0>)

## 一些值得注意的

### `nn.Parameter()`的用法

* `nn.Parameter()`用于自定义可学习的参数，是tensor的子类。

    当神经网络继承自nn.Module时，如果其中存在一些参数是需要纳入模型本身的参数里参与反向传播过程的，则可以使用该方法创建参数。

* 创建方法即为使用`nn.Parameter()`方法包裹tensor:

    `self.weight = nn.Parameter(torch.ones(self.input_dim))`


# RMS Norm

## 核心思想

![LN](pic/RMS_Norm.png)

RMSNorm 主要是在 LayerNorm 的基础上去掉了减均值这一项，其计算效率更高且没有降低性能。

因为原论文中指出LayerNorm发挥作用主要是其缩放不变性（即除以标准差），而不是平移不变性（即减去均值）


In [15]:
class RMS_Norm(nn.Module):
    def __init__(self, input_dim, eps=1e-5):
        super().__init__()
        self.eps = eps
        self.input_dim = input_dim
        self.weight = nn.Parameter(torch.ones(self.input_dim))
    def forward(self, x):
        rms = torch.sqrt(x.pow(2).mean(dim=-1, keepdim=True) + self.eps)
        return x * self.weight / rms


## 代码测试

In [ ]:
x = torch.tensor([[1., 2., 3., 4.], [10., 20., 30., 40.]], dtype=torch.float32)
rmsn = RMS_Norm(4)
x_norm = ln(x)
x_norm

tensor([[-1.3416, -0.4472,  0.4472,  1.3416],
        [-1.3416, -0.4472,  0.4472,  1.3416]], grad_fn=<AddBackward0>)

# Batch Norm

# MLP


# DecoderLayer

# EncoderLayer